## context for may 15 session

- we find it a little difficult to find datasets when we need them, so we want to build a tool to fix this
- we want smart search on dataset descriptions
- "smart" part of smart search will come from AI
- last class we used AI in a very rudimentary way - and it seemed to do what we need it to do. 
- however the approach we took does not scale (we can't copy paste ALL dataset descriptions into the context window of GPT 4 every time we want it to find us a dataset)

### the "real" approach

- we're going to use embeddings

In [1]:
import pandas as pd
import numpy as np
import tiktoken
import openai
from dotenv import load_dotenv,find_dotenv
import os

_ = load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

openai.api_key = api_key

In [3]:
def get_embedding(text, model="text-embedding-3-small"): # 
    text = text.replace("\n", " ")
    response = openai.Embedding.create(input=[text], model=model)

    return response.data[0].embedding

In [7]:
df = pd.read_csv('/Users/arad/repos/pp_kaggle_query/csvs/DatasetVersions.csv')
df = df.loc[~df['Description'].isna(),:].sample(100)
df = df[['DatasetId','Description']]

In [9]:
embedding_encoding = "cl100k_base"
max_tokens = 8000 # technically max_tokens is 8191?
encoding = tiktoken.get_encoding(embedding_encoding)
# if too long remove description - we know we have a few VERY long ones (unlikely to be in this sample)
df["n_tokens"] = df['Description'].apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens]

In [11]:
df["embedding"] = df['Description'].apply(lambda x: get_embedding(x))

In [33]:
test_vector = np.zeros([1536])
dataset_vector = np.array(df.iloc[6,:]['embedding'])
np.linalg.norm(dataset_vector - test_vector)

# the above was just one distance calculation
# when we compare the user query to the datasets, we need to do many calculations
# use the apply method for dataframes

df['distance'] = df['embedding'].apply(lambda x: np.linalg.norm(x - test_vector))
df['distance'].hist()

In [ ]:
def process_user_query(user_query,dataset_embeddings):

    user_embedding = get_embedding(user_query)
    # user embedding and descr embeddings can't both be lists
    user_embedding = np.array(user_embedding)

    # take the distance between user_embedding and all dataset embeddings - dist = numpy.linalg.norm(a-b)

    dataset_embeddings['distance'] = df['embedding'].apply(lambda x: np.linalg.norm(x - user_embedding))

    dataset_embeddings.sort_values(by='distance', ascending=True, inplace=True)

    return df.iloc[0:5,:]['DatasetId'].values

def get_dataset_info(top_5_datasets_array,dataset_df):

    subset_df = dataset_df.loc[dataset_df['DatasetId'].isin(top_5_datasets_array),["Title","Subtitle","Description"]]

    for index, row in subset_df.iterrows():
        print(row['Title'])
        print(row["Subtitle"])
        print("*"*100)
        print("\n")
        print("Dataset Description:\n")
        print(row['Description'])
        
test_query = "i am really interested in doing an analysis on indian crop yields."

test_array = process_user_query(test_query,df)

#get_dataset_info(test_array,df_full)

In [ ]:
# which dataset descriptions are not worthy of being embedded?
df_full = pd.read_csv('/Users/arad/repos/pp_kaggle_query/csvs/DatasetVersions.csv')
df_full.info()
# how much will cost to embed the worthy ones (we'll worry about this later)

In [ ]:
df_full.dropna(inplace = True)
df_full.duplicated(subset=['DatasetId']).sum()
print(df_full['DatasetId'].nunique())
df_full.info()

In [8]:
# deduplicate by datasetid, taking most recent
df_full["CreationDate"] = pd.to_datetime(df_full["CreationDate"]) # CreationDate::date
df_full.sort_values(by = "CreationDate",ascending=False,inplace=True)
df_full.drop_duplicates(subset = ["DatasetId"],keep='first',inplace=True)

In [12]:
# duplicate descriptions
df_full.duplicated(subset=['Description']).sum()

1077

In [24]:
# is user id 6047420
# guillemservera?
users = pd.read_csv('/Users/arad/repos/pp_kaggle_query/csvs/Users.csv')
users.loc[users['Id'] == 6047420,:] # yes!

In [27]:
df_full['char_count'] = df_full['Description'].apply(len)
df_full['char_count'].describe()

In [35]:
# closer inspection - these all seem to be junk
df_full.loc[df_full['char_count'] < 10,:].sort_values(by = 'char_count',ascending=False).to_csv('short_descriptions.csv')